# Script to download and merge the needed data
- First, the AIS data are downloaded for the whole year 2020 
- Second, corresponding to the datapoints received from the AIS data CMEMS wave and physics data are downloaded 
- Third, all parts of AIS data, CMEMS wave and CMEMS phyiscs we are interested in is merged into a single datafram/ csv-file 

- some of the chunks are using code produced by 52North (https://github.com/52North/MariGeoRoute/blob/b3018578ec7f6a128b63c6a44d4514e8fa767cae/AIS/preprocessing_AIS.ipynb) 



---
### Download + Subset of AIS data 


In [ ]:
!pip install wget # to be able to download via ftp 

  Created wheel for wget: filename=wget-3.2-cp37-none-any.whl size=9675 sha256=afe0b8e9c11cd5986ba41f94115b781dd07975b0555690e38cfd3bd1fa159adc
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [ ]:
# Import of all needed libraries 
import wget
import os
import pandas as pd 
from os import walk
from glob import glob
import numpy as np
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
import zipfile
from datetime import datetime, timedelta
from pathlib import Path

In [ ]:
def check_dir(year):
    """
    Function to check how far the download proceeded. 
    
    Parameters
    ----------
    year : int
        describes the year of the data  
    """
    history =[]
    for path, dirs, files in os.walk(str(year)):
        for file in files:
            if file.endswith('.csv'):
                history.append(file.split('.')[0])
    return history

In [ ]:
def download_AIS(year, zones):
    """
    Function to download the AIS data from https://coast.noaa.gov/. 
        
    Parameters
    ----------
    year : int
        datetime as a string as the format YYYY-mm-ddTHH:MM:SSZ
    zones : list 
        describes the UTM zones for which data gets downloaded 
    """
    Path(str(year)).mkdir(parents=True, exist_ok=True)
    resume_download = check_dir(str(year))
    url = "https://coast.noaa.gov/htdata/CMSP/AISDataHandler/{0}/".format(year)
    html_text = requests.get(url).text
    soup = BeautifulSoup(html_text, 'html.parser')
    files = []
    for a in soup.find_all('a', href=True):
         if a.text and a.text.endswith('zip'):
            name, _ = a['href'].split('.')
            l = name.split('_')
            l.append(a.text)
            files.append(l) 

    df = pd.DataFrame(files)
    df.columns = [*df.columns[:-1], 'Files']
  
    for c in df.columns:
        if c == 'Files': continue
        unique_col = len(df[c].unique())
        if unique_col == 12 or unique_col == 6: # some years provid data only for 6 month
            df['Month'] = df[c]
        elif unique_col == 31:
            df['Days'] = df[c]
        elif 20 >= unique_col >= 18 or 'zone' in df[c][0].lower():
            if 'zone' in df[c][0].lower():
                df['Zone'] = pd.to_numeric([z[z.lower().find('zone')+4:]for z in df[c]])
            else:
                df['Zone'] = pd.to_numeric(df[c])
        del df[c]
    
    #  download
    if 'Zone' in df.columns:
        dl = df[df['Zone'].isin(zones)][['Files','Zone']]
    else:
        dl = df[['Files','Month']]
    for file, zone in tqdm(dl.values.tolist()):
        output = '%s_%s' % (year, str(zone))
        Path(str(year)).joinpath(output).mkdir(parents=True, exist_ok=True)
        if file.split('.')[0] in resume_download: continue
        print(file)
        wget.download(os.path.join(url,file)) 
        with zipfile.ZipFile(file, 'r') as zip_ref:
            zip_ref.extractall(os.path.join(str(year), output))
        os.remove(file)

In [ ]:
def subset_AIS_to_CSV(year):
    """
    Function to filter the AIS data on the data we need. 
    Therefore the rows MMSI, VesselName, CallSign, Cargo and TranscieverClass are dropped.
    In addition, the data is filtered to ships where cargo is being transported and those ships are underway. 
    On top of that, the data is roughly filtered to ships that are Panamax 2 size, meaning they fit through the Panama Canal extension.  

    Parameters
    ----------
    year : int
        datetime as a string as the format YYYY-mm-ddTHH:MM:SSZ 
    """
    data_list = []
    for path, dirs, files in os.walk(str(year)):
        for file in files:
                if file.endswith('.csv'):
                    x = os.path.join(path, file)

                    df = pd.read_csv(x)
                    df = df.drop(['MMSI', 'VesselName', 'CallSign', 'Cargo', 'TranscieverClass'], axis=1, errors='ignore')
                    df = df.dropna()
                    df = df.query('(Status == "under way using engine" or Status == 8) & (VesselType in [1016, 1003, 1004] or 79 >= VesselType >= 70) & SOG > 7 & Length < 367 & 30 < Width < 50 & Draft < 16')
                    df['BaseDateTime'] = pd.to_datetime(df.BaseDateTime, format='%Y-%m-%dT%H:%M:%S')
                    data_list.extend(df.values)

    df = pd.DataFrame(data_list, columns=['BaseDateTime', 'LAT', 'LON', 'SOG', 'COG', 'Heading',  'IMO', 'VesselType', 'Status', 'Length','Width', 'Draft'])
    df = df.dropna()
    df = df[df['BaseDateTime'] >= datetime(2016,3,1,3)]
    df.to_csv(os.path.join(str(year), '%s_AIS.csv' % str(year)))
    return df

In [ ]:
def show_plotly(dataframe):
    """
    Function to plot the downloaded data points.  
        
    Parameters
    ----------
    dataframe : pandas DataFrame
        dataframe with the point data and needed SOG 
    """
    import plotly.express as px
    fig = px.scatter_mapbox(dataframe,
                        lat=dataframe.LAT,
                        lon=dataframe.LON, color='SOG', mapbox_style="stamen-toner")
    fig.update_geos(
        lataxis_range=[dataframe.LAT.min(),dataframe.LAT.max()], lonaxis_range=[dataframe.LON.min(), dataframe.LON.max()]
    )
    fig.show()

In [ ]:
# Here the AIS data gets downloaded for the year 2020, and the UTM zones 1 - 25 (https://upload.wikimedia.org/wikipedia/commons/e/ed/Utm-zones.jpg). 
download_AIS(2020, zones=list(range(1,25)))

  0%|          | 0/366 [00:00<?, ?it/s]

AIS_2020_01_01.zip


  0%|          | 1/366 [00:20<2:05:31, 20.64s/it]

AIS_2020_01_02.zip


  1%|          | 2/366 [00:41<2:06:21, 20.83s/it]

...

 27%|██▋       | 100/366 [38:22<1:39:42, 22.49s/it]

AIS_2020_04_10.zip


 28%|██▊       | 101/366 [38:43<1:37:23, 22.05s/it]

AIS_2020_04_11.zip


OSError: ignored

In [ ]:
# Here the data get subsetted as specified above for the year 2020
subset_AIS_to_CSV(year='2020')

2020/2020_01/AIS_2020_01_25.csv


/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning:

elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



2020/2020_01/AIS_2020_01_19.csv


/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning:

elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison



2020/2020_01/AIS_2020_01_20.csv


...

2020/2020_01/AIS_2020_01_16.csv


/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning:

elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison



2020/2020_01/AIS_2020_01_06.csv


/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning:

elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison



,BaseDateTime,LAT,LON,SOG,COG,Heading,IMO,VesselType,Status,Length,Width,Draft
0,2020-01-25 19:00:00,30.68622,-88.03597,5.3,182.9,182.0,IMO9176606,70.0,8.0,228.0,32.0,10.0
1,2020-01-25 20:00:00,30.52765,-88.02257,12.0,175.0,174.0,IMO9176606,70.0,8.0,228.0,32.0,10.0
2,2020-01-25 21:00:00,30.32924,-88.02809,12.5,187.0,187.0,IMO9176606,70.0,8.0,228.0,32.0,10.0
3,2020-01-25 22:00:00,30.12015,-88.05660,13.6,136.4,136.0,IMO9176606,70.0,8.0,228.0,32.0,10.0
4,2020-01-25 23:00:00,29.88354,-87.83000,17.0,144.4,145.0,IMO9176606,70.0,8.0,228.0,32.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...
181,2020-01-22 20:00:00,28.10118,-90.09590,11.5,40.9,44.0,IMO9484261,70.0,8.0,261.0,32.0,11.0
182,2020-01-22 21:00:00,28.25850,-89.93973,11.9,37.8,39.0,IMO9484261,70.0,8.0,261.0,32.0,11.0
183,2020-01-22 22:00:00,28.39628,-89.81469,11.8,38.9,40.0,IMO9484261,70.0,8.0,261.0,32.0,11.0
184,2020-01-22 23:00:00,28.46866,-89.74957,11.9,38.5,40.0,IMO9484261,70.0,8.0,261.0,32.0,11.0


In [ ]:
# Here all subsetted datapoints are shown by the show_plotly function 
year = 2020
df = pd.read_csv(Path(str(year),'%s_AIS.csv' % year))
show_plotly(df)



---
### Download of CMEMS physics and wave data + Merge with AIS data 


In [ ]:
# Import of all needed libraries 
!pip install motuclient
import pandas as pd
import requests
import numpy as np
from motu_utils.utils_cas import authenticate_CAS_for_URL
from motu_utils.utils_http import open_url
import xarray as xr
from datetime import datetime, timezone, timedelta 
import time
from pathlib import Path
from bs4 import BeautifulSoup
import sys

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
# utils to convert dates 
str_to_date = lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S')
date_to_str = lambda x: x.strftime('%Y-%m-%dT%H:%M:%SZ')
str_to_date2 = lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%SZ')

In [ ]:
# input CMEMS credentials
UN_CMEMS = input('Please input your CMEMS username: ')
PW_CMEMS = input('Please input your CMEMS password: ')

Please input your CMEMS username: jj
Please input your CMEMS password: jjj


In [ ]:
def create_request_url(date, lat, lon, datatype):
    """
    Function creates depending on date, latitute and longitude a valid url to request the cmems data using the credentials defined above. 
    Either for the cmems physics or the wave data. 
        
    Parameters
    ----------
    date : str
        datetime as a string as the format YYYY-mm-ddTHH:MM:SSZ
    lat : str, float
        latitude
    lon : str, float
        longitude
    datatype : str 
        describes the type of downloaded data either cmems physics or wave data. 
    """
    if (datatype == 'physics'):
        base_url = 'http://nrt.cmems-du.eu/motu-web/Motu?action=productdownload&service=GLOBAL_ANALYSIS_FORECAST_PHY_001_024-TDS&product=global-analysis-forecast-phy-001-024'
    else:
        base_url = 'http://nrt.cmems-du.eu/motu-web/Motu?action=productdownload&service=GLOBAL_ANALYSIS_FORECAST_WAV_001_027-TDS&product=global-analysis-forecast-wav-001-027'

    y_lo=float(lat)
    y_hi=float(lat)
    x_lo=float(lon)
    x_hi=float(lon)

    if (datatype == 'physics'):
        # depth
        z_hi = 0.5
        z_lo = 0.49

        t_lo=date_to_str(str_to_date2(date).replace(hour=12, minute=0, second=0))
        t_hi=t_lo
        
        url = base_url + '&x_lo={0}&x_hi={1}&y_lo={2}&y_hi={3}&t_lo={4}&t_hi={5}&z_lo={6}&z_hi={7}&mode=console'.format(x_lo, x_hi,y_lo,y_hi,t_lo,t_hi,z_lo,z_hi)
    else:
        dataset_temporal_resolution = 180
        time_in_min = (str_to_date2(date).hour * 60) + str_to_date2(date).minute
        rest = time_in_min % dataset_temporal_resolution
        t_lo = date_to_str((str_to_date2(date) - timedelta(minutes=rest)).replace(second=0))
        t_hi = t_lo

        url = base_url+'&x_lo={0}&x_hi={1}&y_lo={2}&y_hi={3}&t_lo={4}&t_hi={5}&mode=console'.format(x_lo, x_hi, y_lo, y_hi, t_lo, t_hi)
    
    return authenticate_CAS_for_URL(url , UN_CMEMS, PW_CMEMS)

In [ ]:
def retrive_and_interpolate_data(date, lat, lon, dataset_temporal_resolution, datatype):
    """
    Function to retrive all variables from dataset for a specific timestamp, latitude, longitude concidering the temporal resolution of the dataset to calculate interpolated values. 
    To get a higher number of data faster, only the next lower data point is searched, not also the higher one. Accordingly, some lines of code are commented out. 

    Parameters
    ----------
    date : datetime object
        datetime as a date object
    lat : str, float
        latitude
    lon : str, float
        longitude
    dataset_temporal_resolution: int
        the temporal resolution of the dateset (in hours)
    datatype : str 
        describes the type of downloaded data either cmems physics or wave data. 
    """
    h = date.hour 
    rest = h % dataset_temporal_resolution 
    if rest == 0:
        url = create_request_url(date_to_str(date), lat, lon, datatype)
        date = xr.open_dataset(open_url(url).read())
        return np.ravel(date.to_dataframe().reset_index(drop=True).values)
    else:
        nearest_lower = date - timedelta(hours= rest)
        #nearest_upper = date + timedelta(hours=dataset_temporal_resolution-rest)
        
        url_lower = create_request_url(date_to_str(nearest_lower), lat, lon, datatype)
        #url_upper = create_request_url(date_to_str(nearest_upper), lat, lon, datatype)
        
        bytes_lower = open_url(url_lower).read()
        #bytes_upper = open_url(url_upper).read()
        
        try:
            data_lower = xr.open_dataset(bytes_lower)
            #data_upper = xr.open_dataset(bytes_upper) 
        except:
            # print the error tag from html
            print(BeautifulSoup(bytes_lower, 'html.parser').find('p', {"class": "error"}))
            #print(BeautifulSoup(bytes_upper, 'html.parser').find('p', {"class": "error"}))
       
      
        v_lower = data_lower.to_dataframe().reset_index(drop=True)
        #v_upper = data_upper.to_dataframe().reset_index(drop=True)
    
        return np.ravel(v_lower)
    
        # temporal interpolation 
        alpha = rest / dataset_temporal_resolution
        return np.ravel((1- alpha)* v_lower.values + (alpha * v_upper.values))

In [ ]:
def append_enviorment_data(year):
    """
    Function which merges the downloaded data into one dataframe.  
        
    Parameters
    ----------
    year : int
        datetime as a string as the format YYYY-mm-ddTHH:MM:SSZ 
    """
    src_csv_path = Path(str(year),'%s_AIS.csv' % year)
    output_csv_path = Path(str(year),'%s_merged.csv' % year)
    
    # get extracted AIS data and remove index column
    df = pd.read_csv(src_csv_path,parse_dates=['BaseDateTime'], date_parser=str_to_date)
    df.drop(['Unnamed: 0'], axis=1, errors='ignore', inplace=True)
    
    # define new columns for the output datafarme 
    cols = list(df.columns) + ['VHM0_WW', 'VMDR_SW2', 'VMDR_SW1','VMDR', 'VTM10', 'VTPK', 'VPED', 'VTM02', 'VMDR_WW', 'VTM01_SW2', 'VHM0_SW1', 'VTM01_SW1', 'VSDX', 'VSDY', 'VHM0', 'VTM01_WW', 'VHM0_SW2'] + ['thetao', 'so', 'uo', 'vo', 'zos', 'mlotst', 'bottomT', 'siconc', 'sithick', 'usi', 'vsi']
    
    # check if already appended data to resume in case of disconnetion or other errors
    data_list = []
    if Path(output_csv_path).exists():
        data_list = list(pd.read_csv(output_csv_path).drop(['Unnamed: 0'], axis=1, errors='ignore').values)
        print('Resuming download from row %s ' % len(data_list))
        
    # loop over the AIS data starting from the last index, where it has stopped
    last_index = len(data_list)
    for x in df.values[last_index:]:
        date, lat, lon = x[:3]

        env_variables = retrive_and_interpolate_data(date, lat, lon, 3, 'wave')

        phy_variables = retrive_and_interpolate_data(date, lat, lon, 3, 'physics')
        data_list.append(np.concatenate([x, env_variables, phy_variables]))
        
        pd.DataFrame(data_list, columns=cols).to_csv(output_csv_path)
        last_index+=1
        sys.stdout.write("\rEntry row index: %s/%s" % (last_index, len(df)))
        sys.stdout.flush()
        
    return pd.DataFrame(data_list, columns=cols)

In [ ]:
# Here the download and merge is ultimately performed for the year 2020 
result = append_enviorment_data(2020)

Resuming download from row 75 
http://nrt.cmems-du.eu/motu-web/Motu?action=productdownload&service=GLOBAL_ANALYSIS_FORECAST_WAV_001_027-TDS&product=global-analysis-forecast-wav-001-027&x_lo=-122.44421000000001&x_hi=-122.44421000000001&y_lo=47.71676&y_hi=47.71676&t_lo=2020-11-14T21:00:00Z&t_hi=2020-11-14T21:00:00Z&mode=console
http://nrt.cmems-du.eu/motu-web/Motu?action=productdownload&service=GLOBAL_ANALYSIS_FORECAST_WAV_001_027-TDS&product=global-analysis-forecast-wav-001-027&x_lo=-122.44421000000001&x_hi=-122.44421000000001&y_lo=47.71676&y_hi=47.71676&t_lo=2020-11-15T00:00:00Z&t_hi=2020-11-15T00:00:00Z&mode=console
http://nrt.cmems-du.eu/motu-web/Motu?action=productdownload&service=GLOBAL_ANALYSIS_FORECAST_PHY_001_024-TDS&product=global-analysis-forecast-phy-001-024&x_lo=-122.44421000000001&x_hi=-122.44421000000001&y_lo=47.71676&y_hi=47.71676&t_lo=2020-11-14T12:00:00Z&t_hi=2020-11-14T12:00:00Z&z_lo=0.49&z_hi=0.5&mode=console
http://nrt.cmems-du.eu/motu-web/Motu?action=productdownload

This completes the download and merging of the data and the data can now be used to build the model. 

In [ ]:
# This is an optional script for merging the data if you produce data on two different instances.
a = pd.read_csv("input1")
b = pd.read_csv("input2")
#b = b.dropna(axis=1)

merged = a.append(b)
merged
merged.to_csv("output", index=False)